Import dos pacotes

In [2]:
# Processamento dos dados
import pandas as pd

# Visualização dos dados
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento dos dados
from sklearn.preprocessing import StandardScaler

# Modelos de Agrupamento
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# Métricas
from sklearn.metrics import silhouette_score

# Limpeza de memória
import gc

# configs do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-rqlwc27m because the default path (/home/mbaltar/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
metadados = pd.read_excel('metadata.xlsx')
metadados

,Feature,Feature_Type,Description
0,age,numeric,age of a person
1,job,"Categorical,nominal","type of job ('admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')"
2,marital,"categorical,nominal","marital status ('divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)"
3,education,"categorical,nominal","('basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')"
4,default,"categorical,nominal","has credit in default? ('no','yes','unknown')"
5,housing,"categorical,nominal","has housing loan? ('no','yes','unknown')"
6,loan,"categorical,nominal","has personal loan? ('no','yes','unknown')"
7,contact,"categorical,nominal","contact communication type ('cellular','telephone')"
8,month,"categorical,ordinal","last contact month of year ('jan', 'feb', 'mar', …, 'nov', 'dec')"
9,dayofweek,"categorical,ordinal","last contact day of the week ('mon','tue','wed','thu','fri')"


In [4]:
dados = pd.read_csv('new_train.csv', sep=',')
dados.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,no


Removendo a target para trabalharmos com a base em um problema não supervisionado

In [5]:
explicativas = dados.drop(columns='y', axis=1).copy()
explicativas.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'],
      dtype='object')

Separando as explicativas categóricas e numéricas

In [6]:
expl_cat = explicativas[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']].copy()
expl_num = explicativas[['age', 'duration', 'campaign', 'pdays', 'previous']]

In [7]:
expl_num.isnull().sum()

age         0
duration    0
campaign    0
pdays       0
previous    0
dtype: int64

Transformação dos dados com padronização

Trazendo as variáveis para uma mesma escala

z = (x - u) / s

onde 'u' é a média na amostra de treinamento, 's' é o desvio padrão da amostra


In [8]:
scale = StandardScaler()
expl_num_scale = scale.fit_transform(expl_num)
expl_num_scale

array([[ 0.86373877, -0.12019627,  0.52298128,  0.19658384, -0.35012691],
       [-0.28972159, -0.2167318 , -0.20368791,  0.19658384,  1.65381294],
       [ 3.65126795,  3.43617293, -0.56702251,  0.19658384, -0.35012691],
       ...,
       [ 1.34434725, -0.49089273,  0.52298128,  0.19658384, -0.35012691],
       [-1.05869515, -0.3596044 , -0.56702251,  0.19658384, -0.35012691],
       [-0.48196498,  1.10387435,  0.15964669,  0.19658384, -0.35012691]])

O algoritmo

In [9]:
# Vamos usar dois algoritmos: KMeans e Aglomerativo para compará-los
km = KMeans(n_clusters=5, random_state=42)
agg = AgglomerativeClustering(n_clusters=5)

Rodar o agrupamento nos dados

In [10]:
explicativas['kmeans_05'] = km.fit_predict(expl_num_scale)
explicativas.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,kmeans_05
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,1
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,0
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,3
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,0
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,1


In [11]:
explicativas['kmeans_05'].value_counts()

0    17407
1    10086
3     2881
4     1339
2     1237
Name: kmeans_05, dtype: int64

In [12]:
# Para o algoritmo Aglomerativo agora
explicativas['agg_05'] = agg.fit_predict(expl_num_scale)

: 

: 

Não consegui executar do código acima em diante porque não teve memória suficiente no meu computador. Partindo para o Google Collab